# Plan and Execute for Deep Web Search 

- This code is designed to plan and execute a search on the deep web using a specified search engine.


In [1]:

import sys
import os
from openai import AzureOpenAI
import requests
import json
from urllib.parse import urljoin
from langchain_core.prompts import PromptTemplate
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from langchain_core.prompts import load_prompt
import sys
parent_dir = os.path.dirname(os.path.dirname(os.getcwd()))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from utils.search_utils import web_search, url_search, extract_contexts_async
from IPython.display import Markdown, display
from datetime import datetime
import pytz
import time
import os
from dotenv import load_dotenv
load_dotenv(override=True) 
import logging

logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


# Get credentials from environment variables
BING_GROUNDING_PROJECT_ENDPOINT = os.getenv("BING_GROUNDING_PROJECT_ENDPOINT")
BING_GROUNDING_CONNECTION_ID = os.getenv("BING_GROUNDING_CONNECTION_ID")
BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME")
BING_GROUNDING_MAX_RESULTS = int(os.getenv("BING_GROUNDING_MAX_RESULTS", 10))
BING_GROUNDING_MARKET = os.getenv("BING_GROUNDING_MARKET", "ko-KR")
BING_GROUNDING_SET_LANG = os.getenv("BING_GROUNDING_SET_LANG", "ko-KR")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "gpt-4o-mini")

# Web search mode: "google" or "bing"
web_search_mode = "bing"


client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-08-01-preview"
)

current_dir = os.getcwd()

def rewrite_query_for_search(query, client: AzureOpenAI):
    
  prompt_path = os.path.join(current_dir, "prompts", "rewrite_prompt.yaml")
  QUERY_REWRITE_PROMPT = load_prompt(prompt_path, encoding="utf-8")
  response = client.chat.completions.create(
      model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
      messages=[
          {"role": "system", "content": QUERY_REWRITE_PROMPT.format(
            user_query=query)},
          {"role": "user", "content": query}
      ],
      temperature=0.8,
      max_tokens=300,
      response_format= {"type": "json_object"},
  )
  
  return json.loads(response.choices[0].message.content.strip())
    
  
def plan_query_for_search(query, client: AzureOpenAI):

  current_date = datetime.now(tz=pytz.timezone("Asia/Seoul")).strftime("%Y-%m-%d")
  
  prompt_path = os.path.join(current_dir, "prompts", "planner_prompt.yaml")
  PLANNER_PROMPT = load_prompt(prompt_path, encoding="utf-8")
  
  response = client.chat.completions.create(
      model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
      messages=[
          {"role": "system", "content": PLANNER_PROMPT.format(current_date=current_date, query=query)},
          {"role": "user", "content": query}
      ],
      temperature=0.8,
      max_tokens=300,
      response_format= {"type": "json_object"},
  )
  
  return json.loads(response.choices[0].message.content.strip())


In [2]:


   

# 웹 검색 결과를 활용해 LLM 답변을 생성하는 비동기 함수
async def process_deep_search(max_result_count=3, input_query=None, web_search_mode=None, product_name=None):
    
    start_time = time.time()
    print(f"Original Input: {input_query}")

    # 검색 모드가 지정되지 않으면 환경변수
    if web_search_mode is None:
        web_search_mode = os.getenv("WEB_SEARCH_MODE", "google").lower()
    print(f"############## Web Search Mode: {web_search_mode}")
    
    # query rewrite (검색용/LLM용)
    query_rewrite = rewrite_query_for_search(input_query, client)
    print(f"Web Search Query: {query_rewrite['search_query']}")
    print(f"LLM Query: {query_rewrite['llm_query']}")

    # plan deep query
    search_plan_json = plan_query_for_search(query_rewrite['search_query'], client)
    print(f"Planned Search Queries: {search_plan_json}")
    search_queries = search_plan_json.get("search_queries", [])
    all_contexts = []
    
    for i, query in enumerate(search_queries):
        print(f"Processing query {i+1}: {query}")
        search_results = url_search(query, max_result_count, web_search_mode=web_search_mode, product_name=product_name)
        
        print(f"Search Results: {len(search_results)} results found for query '{query}'")
        
        print("Analyze search results...")        
        if search_results:
            url_snippet_tuples = [(r["link"], r["snippet"]) for r in search_results]
            contexts = await extract_contexts_async(url_snippet_tuples=url_snippet_tuples)
            
            formatted_contexts = [
                f"[search keyword: {query}]\n{context}"
                for context in contexts
            ]
            
            all_contexts.extend(formatted_contexts)
        
    
    
    current_date = datetime.now(tz=pytz.timezone("Asia/Seoul")).strftime("%Y-%m-%d")
        
    # Get the current working directory for this notebook
    prompt_path = os.path.join(current_dir, "prompts", "generate_prompt.yaml")
    GENERATE_PROMPT = load_prompt(prompt_path, encoding="utf-8")
    
    print("Generate response...")            
    
    answer_messages = [
        {"role": "system", "content": GENERATE_PROMPT.format(
            product_name=product_name,
            date=current_date,
            contexts=all_contexts,
            user_query=query_rewrite['llm_query'],
        )},
        {"role": "user", "content": query_rewrite['llm_query']}
    ]
    
    response = client.chat.completions.create(
        model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
        messages=answer_messages,
        top_p=0.9,
        max_tokens=1500
    )
    display(Markdown(response.choices[0].message.content))
    
    end_time = time.time()
    print(f"elapsed time: {end_time - start_time:.2f} seconds\n\n")


In [3]:
RESULTS_COUNT = 5

inputs = [
    "갤럭시 s25와 아이폰4 비교해줘"
]

web_search_mode = "bing"

for input in inputs:
    print(f"Bing Grounding 검색 사용: {input}")
    await process_deep_search(max_result_count=RESULTS_COUNT, input_query=input, web_search_mode=web_search_mode, product_name="삼성전자")  # product_name은 필요에 따라 변경 가능

Bing Grounding 검색 사용: 갤럭시 s25와 아이폰4 비교해줘
Original Input: 갤럭시 s25와 아이폰4 비교해줘
############## Web Search Mode: bing


Web Search Query: 갤럭시 S25 아이폰4 비교
LLM Query: 갤럭시 S25와 아이폰4의 주요 사양, 성능, 디자인, 기능 등을 비교해 주세요. 각 모델의 장단점도 함께 설명해 주시면 좋겠습니다.
Planned Search Queries: {'search_queries': ['갤럭시 S25 아이폰4 비교', '갤럭시 S25 리뷰', '아이폰4 사양']}
Processing query 1: 갤럭시 S25 아이폰4 비교
Search Results: 3 results found for query '갤럭시 S25 아이폰4 비교'
Analyze search results...
Processing query 2: 갤럭시 S25 리뷰
Search Results: 0 results found for query '갤럭시 S25 리뷰'
Analyze search results...
Processing query 3: 아이폰4 사양


2025-06-23 12:35:46,792 - utils.search_utils - ERROR - Request failed: Client error '403 Forbidden' for url 'https://namu.wiki/w/iPhone%204'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403


Search Results: 3 results found for query '아이폰4 사양'
Analyze search results...
Generate response...


갤럭시 S25와 아이폰4는 시대가 다른 스마트폰으로, 각기 다른 특성과 장점을 가지고 있습니다. 아래는 두 모델의 주요 사양, 성능, 디자인, 기능에 대한 비교입니다.

| **항목**            | **갤럭시 S25**                       | **아이폰4**                          |
|-------------------|-----------------------------------|-----------------------------------|
| **출시 연도**      | 2025년                             | 2010년                             |
| **디스플레이**     | 6.8인치 AMOLED, 120Hz, HDR10+    | 3.5인치 Retina, 960x640 해상도     |
| **프로세서**       | 최신 엑시노스 또는 스냅드래곤      | A4 칩셋 (싱글코어)                  |
| **RAM**           | 12GB 이상                          | 512MB                             |
| **저장 용량**      | 128GB, 256GB, 512GB               | 16GB, 32GB                        |
| **카메라**         | 후면 쿼드 카메라(최대 200MP)      | 후면 500만 화소, 전면 VGA          |
| **배터리**         | 5000mAh 이상, 고속 충전 지원      | 1420mAh, 고속 충전 불가           |
| **운영체제**       | Android 13 (업데이트 가능)        | iOS 4 (업데이트 제한적)           |
| **기타 기능**      | 5G 지원, IP68 방수/방진           | 페이스타임, 비디오 통화 지원      |

### **성능 및 기능 비교**
- **갤럭시 S25**
  - **장점**:
    - 최신 하드웨어와 소프트웨어를 탑재하여 매우 빠르고 쾌적한 사용자 경험을 제공.
    - 고해상도 디스플레이와 쿼드 카메라로 뛰어난 사진 및 영상 촬영 가능.
    - 방수 및 방진 기능을 제공하여 다양한 환경에서 안전하게 사용할 수 있음.
  - **단점**:
    - 고가의 제품군으로 가격 부담이 있을 수 있음.

- **아이폰4**
  - **장점**:
    - 당시 기준으로 혁신적인 디자인과 레티나 디스플레이가 높은 해상도를 제공.
    - 간편한 iOS 운영체제로 사용이 직관적이며, 애플 생태계와의 연동성이 뛰어남.
  - **단점**:
    - 구형 하드웨어로 인해 최신 앱 및 기능에 대한 호환성이 낮고, 성능이 떨어짐.
    - 배터리 용량과 충전 속도가 부족하여 장시간 사용에 불편할 수 있음.

### **디자인 비교**
- **갤럭시 S25**: 현대적이고 세련된 디자인을 가지고 있으며, 다양한 색상 옵션과 프리미엄 소재를 사용하여 고급스러운 느낌을 준다.
- **아이폰4**: 강한 스테인리스 스틸 프레임과 강화유리로 이루어진 클래식한 디자인으로, 여전히 많은 사람들에게 아름다움의 기준으로 여겨진다.

### **결론**
갤럭시 S25는 최신 기술과 뛰어난 성능을 제공하는 현대적인 스마트폰으로, 고급 사용자들에게 적합합니다. 반면 아이폰4는 역사적인 모델로, 당시의 혁신을 경험하고 싶은 사용자에게 좋습니다. 각 모델의 장단점과 용도를 고려하여 선택하는 것이 중요합니다.

elapsed time: 58.71 seconds


